In [9]:
import pandas as pd
import numpy as np
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score ,f1_score,roc_curve
import matplotlib.pyplot as plt


In [10]:
%load_ext autoreload
%autoreload 2
from data_prep import df_prep  
from data_prep import  NLP_Vectorizer
from data_prep import parse_line
from model_src import NLP_model


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[nltk_data] Downloading package wordnet to /Users/nathan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


AttributeError: module 'tensorflow_core._api.v2.train' has no attribute 'Optimizer'

In [ ]:
col_names = ['marketplace','customer_id','review_id','product_id','product_parent','product_title','product_category','star_rating','helpful_votes','total_votes','vine','verified_purchase','review_headline','review_body','review_date']
cols = {}
for i in range(len(col_names)):
    print (str(i)+': '+col_names[i])
    cols[col_names[i]] = i

In [ ]:
np.random.seed(500)
df = pd.read_csv('data/sample_02.csv')
#df = df[df['7']==1]
df = df[df['9']>10]

df = df.sample(frac=.2, random_state=1)
len(df)

In [ ]:
Corpus = df_prep(df,.4,.0)

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus[['text','text_final','help_score','help_votes','stars']],Corpus['label'],test_size=0.3)



Train_X_Vector, Test_X_Vector, Train_Y, Test_Y = vectorize_df(Train_X['text_final'], Test_X['text_final'], Train_Y, Test_Y,'TF_IDF')





In [ ]:
TF_IDF = NLP_Vectorizer('TF_IDF',{'max_features':2000,'ngram_range':(1,3)})
orig_Test_Y = Test_Y
orig_Train_Y = Train_Y
Test_Y = TF_IDF.encode_Y(Test_Y)
Train_Y = TF_IDF.encode_Y(Train_Y)

In [ ]:
TF_IDF.fit(Train_X['text_final'])
Train_X_Vector = TF_IDF.transform(Train_X['text_final'])
Test_X_Vector = TF_IDF.transform(Test_X['text_final'])

In [ ]:
list(Train_X['text_final'])[0] == parse_line(list(Train_X['text'])[0])

In [ ]:
NVB = NLP_model('XGBoost')

In [ ]:
NVB.fit(Train_X_Vector,Train_Y)

In [ ]:
NVB_predict = NVB.predict(Test_X_Vector)

In [ ]:
accuracy_score(NVB_predict,Test_Y)

In [ ]:
NVB_predict

In [ ]:
sum(NVB_predict)/len(NVB_predict)

In [ ]:
sum(Test_Y)/len(Test_Y)

In [ ]:
Test_X['preds'] = NVB_predict
Test_X['actual'] = Test_Y

Test_X['correct'] = Test_X['preds'] == Test_X['actual']

In [ ]:
Test_X['actual']

In [ ]:
plt.hist(Test_X[Test_X['correct']==False]['help_score'],bins=10,normed=True,color='red',alpha=.5)
plt.hist(Test_X[Test_X['correct']==True]['help_score'],bins=10,normed=True,color='blue',alpha=.5)